In [124]:
import missingno as msno
from itertools import permutations, product
import itertools
import calendar
from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from helper_functions import *
import pandas as pd
from pandas import read_csv
import csv
import unittest
import numpy as np
from pprint import pprint
from setuptools import setup, find_packages
from codecs import open
from os import path
from epw import epw
import sys
from datetime import datetime, date, timezone
from pytz import timezone
import pytz
import numpy as np
import pandas as pd
import requests
from epw import epw
import time
from tqdm import tqdm
from pprint import pprint
import pandas as pd
import math
import pandas as pd
from hoboreader import HoboReader
import sys
print(sys.version)
%matplotlib inline
pd.set_option('mode.chained_assignment', None)
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 150


3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)]


In [344]:
# DL1	Stone Garden - Tjaden Hall
# DL2	PSB Parking Lot
# DL3	Olin Library
# DL4	Milstein Hall Roof
# DL5	Game Farm Road

S1 = HoboReader("DL1_Stone_Garden_Tjaden_Hall.csv")
S2 = HoboReader("DL2_PSB_Parking_Lot.csv")
S3 = HoboReader("DL3_Olin_Library.csv")
S4 = HoboReader("DL4_Milstein_Hall_Roof.csv")
S5 = HoboReader("DL5_Game_Farm_Road.csv")

df1 = S1.get_dataframe()
df2 = S2.get_dataframe()
df3 = S3.get_dataframe()
df4 = S4.get_dataframe()
df5 = S5.get_dataframe()

# DF5 (GAME FARM ROAD) HAS TO BE SHIFTED BACK BECAUSE IT NATIVELY RECORDS DST, OUR STATIONS DO NOT.
# THE OTHER STATIONS ARE NOT SHIFTED (HENCE 0 VALUE FOR PERIODS), JUST THERE FOR SANITY

df5 = df5.shift(-2,freq="H",  fill_value=0)
df4 = df4.shift(0,freq="H", fill_value=0)
df3 = df3.shift(0,freq="H", fill_value=0)
df2 = df2.shift(0,freq="H", fill_value=0)
df1 = df1.shift(0,freq="H", fill_value=0)


# Remove time zones (we are only interested in the local timezone)

df5 =df5.tz_convert(None)
df4 =df4.tz_convert(None)
df3 =df3.tz_convert(None)
df2 =df2.tz_convert(None)
df1 =df1.tz_convert(None)

df5 =df5 .tz_localize('UTC')
df4 =df4 .tz_localize('UTC')
df3 =df3 .tz_localize('UTC')
df2 =df2 .tz_localize('UTC')
df1 =df1 .tz_localize('UTC')

df5 = df5.shift(-5,freq="H", fill_value=0)
df4 = df4.shift(-5,freq="H", fill_value=0)
df3 = df3.shift(-5,freq="H", fill_value=0)
df2 = df2.shift(-5,freq="H", fill_value=0)
df1 = df1.shift(-5,freq="H", fill_value=0)



In [316]:
display(df1)

title,#,Date Time,Pressure,Temp,RH,Solar Radiation,MRT,Wind Speed,Gust Speed,Wind Direction,DewPt
timezone_str,NaN,GMT-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,NaN,NaN,in Hg,Â°C,%,W/mÂ²,Â°c,m/s,m/S,Ã¸,Â°C
logger_serial_number,NaN,NaN,20098500,20098500,20098500,20098500,20098500,20098500,20098500,20098500,20098500
sensor_serial_number,NaN,NaN,10751992,10763331,10763331,10775592,20496099,20706789,20706789,20706789,10763331
datetimes,,,,,,,,,,,
2019-09-17 17:37:26+00:00,1,09/17/19 05:37:26 PM,29.3395,19.673889,59.9,5.6,19.936111,1.00137,2.668829,328.0,11.722222
2019-09-17 17:47:26+00:00,2,09/17/19 05:47:26 PM,29.3395,19.483889,53.6,4.4,19.388889,1.00137,2.338019,5.6,9.777778
2019-09-17 17:57:26+00:00,3,09/17/19 05:57:26 PM,29.3454,19.127222,52.0,1.9,18.888889,1.00137,1.671930,4.2,9.111111
2019-09-17 18:07:26+00:00,4,09/17/19 06:07:26 PM,29.3513,18.793889,53.4,0.6,18.461111,0.67056,1.998269,1.4,9.111111
2019-09-17 18:17:26+00:00,5,09/17/19 06:17:26 PM,29.3543,18.508889,54.0,0.6,18.105000,1.00137,2.338019,1.4,9.000000


In [355]:
def plot(variable_id, period_len, day, resample_len, month, year):

    start_time = str(year) + '-' + str(month) + '-' + str(day) + ' 11:00:00+00:00'
    end_time = str(year) + '-' + str(month) + '-' + str(day + period_len ) + ' 04:00:00+00:00'
    utc = timezone('UTC')

    df1_ = df1.loc[start_time:end_time]
    df2_ = df2.loc[start_time:end_time]
    df3_ = df3.loc[start_time:end_time]
    df4_ = df4.loc[start_time:end_time]
    df5_ = df5.loc[start_time:end_time]

    # Resample subhourly data to 1 hourly graph (OUR STATIONS RECORD IN 10MIN INTERVALS, GFR IN 1HR, RESAMPLE FOR EASY COMPARISON. CAN TURN OFF)
    df1_ = df1_.resample(str(resample_len) + 'H').mean()
    df2_ = df2_.resample(str(resample_len) + 'H').mean()
    df3_ = df3_.resample(str(resample_len) + 'H').mean()
    df4_ = df4_.resample(str(resample_len) + 'H').mean()
    df5_ = df5_.resample(str(resample_len) + 'H').mean()

    # Resize figure
    custom_figsize = (30, 15)

    # Change lineweights
    lineweights = 1.5

    variables = [
        "Wind Speed", "MRT", "Temp", "RH", "Solar Radiation", "Wind Speed",
        "Gust Speed", "Wind Direction", "DewPt"
    ]

    variable = variables[variable_id]

    # Plot both temp and mrt when temp is selected

    if variable == "MRT" or variable == "DewPt" or variable == "Gust Speed":
        ax = df1_.plot(y=variable, lw=1, figsize=custom_figsize, color='red')
        df2_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='blue',
                  ax=ax)
        df3_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='green',
                  ax=ax)
        df4_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='aqua',
                  ax=ax)
        df5_.plot(y=variable,
                  lw=3.5,
                  figsize=custom_figsize,
                  color='black',
                  ax=ax)

    elif variable == "Temp":
        # Plot two dataframes into one plot
        ax = df1_.plot(y=variable, lw=2, figsize=custom_figsize, color='red')
        df2_.plot(y=variable,
                  lw=2,
                  figsize=custom_figsize,
                  color='blue',
                  ax=ax)
        df3_.plot(y=variable,
                  lw=2,
                  figsize=custom_figsize,
                  color='green',
                  ax=ax)
        df4_.plot(y=variable,
                  lw=2,
                  figsize=custom_figsize,
                  color='aqua',
                  ax=ax)
        df5_.plot(y=variable,
                  lw=3.5,
                  figsize=custom_figsize,
                  color='black',
                  ax=ax)

        ax = df1_.plot(y="MRT",
                       lw=1,
                       figsize=custom_figsize,
                       style=['--'],
                       color='red',
                       ax=ax)
        df2_.plot(y="MRT",
                  lw=1,
                  figsize=custom_figsize,
                  style=['--'],
                  color='blue',
                  ax=ax)
        df3_.plot(y="MRT",
                  lw=1,
                  figsize=custom_figsize,
                  style=['--'],
                  color='green',
                  ax=ax)
        df4_.plot(y="MRT",
                  lw=1,
                  figsize=custom_figsize,
                  style=['--'],
                  color='aqua',
                  ax=ax)

    else:
        # Plot two dataframes into one plot
        ax = df1_.plot(y=variable, lw=1, figsize=custom_figsize, color='red')
        df2_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='blue',
                  ax=ax)
        df3_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='green',
                  ax=ax)
        df4_.plot(y=variable,
                  lw=1,
                  figsize=custom_figsize,
                  color='aqua',
                  ax=ax)
        df5_.plot(y=variable,
                  lw=2,
                  figsize=custom_figsize,
                  color='black',
                  ax=ax)

    # Legend
    ax.legend([
        "DL1 - Stone Garden - Tjaden Hall", "DL2 - PSB Parking Lot",
        "DL3 - Olin Library", "DL4 - Milstein Hall Roof ",
        "DL5 - Game Farm Road"
    ],
              loc='upper left',
              frameon=False,
              fontsize=20)

    # Fix xticks

    # years = mdates.YearLocator()  # every year
    # months = mdates.MonthLocator()  # every month
    # days = mdates.DayLocator()  # every day
    # minutes = mdates.MinuteLocator()  # every minute
    #days_fmt = mdates.DateFormatter('%d')

    # format the ticks
    # if period_len > 1:
    #    hours = mdates.HourLocator(interval=6)  # every 6 hours
    #    hours_fmt = mdates.DateFormatter('%H:00')
    #    ax.xaxis.set_major_locator(hours)
    #    ax.xaxis.set_major_formatter(hours_fmt)
    #
    # else:
    #    hours = mdates.HourLocator()  # every 6 hours
    #    hours_fmt = mdates.DateFormatter('%H:00')
    #
    #    ax.xaxis.set_major_locator(hours)
    #    ax.xaxis.set_major_formatter(hours_fmt)

    # we dont have minor ticks for now
    # ax.xaxis.set_minor_locator(hours)
    ax.tick_params(axis='both', which='minor', labelsize=22)
    ax.tick_params(axis='both', which='major', labelsize=22)

    # Labels
    plt.ylabel(variable, fontname="Consolas", fontsize=26)
    plt.xlabel("Time", fontname="Consolas", fontsize=26)

    # Title
    ax.set_title(str(
        start_time.split(' ')[0] + " "
        'to'
        " " + end_time.split(' ')[0] + " " + variable),
                 fontname="Consolas",
                 fontsize=28)


# Variable to plot
# 1 = "MRT °C"
# 2 = "Temp °C"
# 3 = "RH"
# 4 = "Solar Radiation"
# 5 = "Wind Speed"
# 6 = "Gust Speed"
# 7 = "Wind Direction"
# 8 = "DewPt"

interact(plot,
         variable_id=(1, 8, 1),
         period_len=(1, 30, 1),
         resample_len=(1, 24, 1),
         day=(1, 31, 1),
         month=(1, 12, 1),
         year=(2019, 2021, 1))

interactive(children=(IntSlider(value=4, description='variable_id', max=8, min=1), IntSlider(value=15, descrip…

<function __main__.plot(variable_id, period_len, day, resample_len, month, year)>

In [317]:
#PRINT RESAMPLED DATA (HOURLY) TO A .CSV FOR EPW FILE

S1 = HoboReader("DL1_Stone_Garden_Tjaden_Hall.csv")
S2 = HoboReader("DL2_PSB_Parking_Lot.csv")
S3 = HoboReader("DL3_Olin_Library.csv")
S4 = HoboReader("DL4_Milstein_Hall_Roof.csv")

df1 = S1.get_dataframe()
df2 = S2.get_dataframe()
df3 = S3.get_dataframe()
df4 = S4.get_dataframe()

df1 = df1.resample('H').mean()
df2 = df2.resample('H').mean()
df3 = df3.resample('H').mean()
df4 = df4.resample('H').mean()

display(df1)
display(df2)
display(df3)
display(df4)

#print(df1)
#print(df2)
#print(df3)
#print(df4)

df1.to_csv('DL1_TJADEN_hourly.csv')
df2.to_csv('DL2_PSB_hourly.csv')
df3.to_csv('DL3_OLIN_hourly.csv')
df4.to_csv('DL4_MILSTEIN_hourly.csv')

title,Pressure,Temp,RH,Solar Radiation,MRT,Wind Speed,Gust Speed,Wind Direction,DewPt
timezone_str,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,in Hg,Â°C,%,W/mÂ²,Â°c,m/s,m/S,Ã¸,Â°C
logger_serial_number,20098500,20098500,20098500,20098500,20098500,20098500,20098500,20098500,20098500
sensor_serial_number,10751992,10763331,10763331,10775592,20496099,20706789,20706789,20706789,10763331
datetimes,,,,,,,,,
2019-09-17 17:00:00-05:00,29.341467,19.428333,55.166667,3.966667,19.404630,1.001370,2.226259,112.600000,10.203704
2019-09-17 18:00:00-05:00,29.352800,18.029444,56.333333,0.600000,17.637037,0.724205,1.671930,61.466667,9.185185
2019-09-17 19:00:00-05:00,29.361167,16.105185,66.033333,0.600000,15.620278,0.275675,0.947725,53.033333,9.768519
2019-09-17 20:00:00-05:00,29.383333,14.576574,73.133333,0.600000,14.133704,0.277165,0.780830,227.600000,9.824074
2019-09-17 21:00:00-05:00,29.401517,13.372870,80.533333,0.600000,12.943333,0.275675,0.669070,234.633333,10.101852


title,Wind Speed,Gust Speed,Wind Direction,MRT,Temp,RH,Solar Radiation,DewPt
timezone_str,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,m/s,m/s,Ã¸,Â°C,Â°C,%,W/mÂ²,Â°C
logger_serial_number,20702489,20702489,20702489,20702489,20702489,20702489,20702489,20702489
sensor_serial_number,20496534,20496534,20496534,20693834,20698453,20698453,20702166,20698453
datetimes,,,,,,,,
2019-09-28 09:00:00-05:00,1.671930,4.009949,184.400000,31.178889,24.797778,57.700000,1276.900000,15.888889
2019-09-28 10:00:00-05:00,1.280770,3.616554,231.833333,33.871204,26.324630,54.450000,584.183333,16.388889
2019-09-28 11:00:00-05:00,1.284495,3.507774,237.450000,37.643889,28.447222,49.966667,733.116667,17.018519
2019-09-28 12:00:00-05:00,1.451390,3.227629,36.583333,36.169074,29.387593,45.550000,480.416667,16.398148
2019-09-28 13:00:00-05:00,1.284495,2.783569,63.600000,33.850000,28.846481,47.116667,351.883333,16.490741


title,MRT,Temp,RH,Solar Radiation,Wind Speed,Gust Speed,Wind Direction,DewPt
timezone_str,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,Â°C,Â°C,%,W/mÂ²,m/s,m/s,Ã¸,Â°C
logger_serial_number,20702490,20702490,20702490,20702490,20702490,20702490,20702490,20702490
sensor_serial_number,20496098,20698454,20698454,20702165,20706788,20706788,20706788,20698454
datetimes,,,,,,,,
2019-09-27 13:00:00-05:00,27.604444,22.405556,39.900000,691.900000,2.004974,5.009083,180.950000,8.083333
2019-09-27 14:00:00-05:00,29.302315,23.516944,38.400000,323.133333,1.339630,3.394524,325.450000,8.546296
2019-09-27 15:00:00-05:00,29.081111,24.074630,35.200000,349.783333,1.114620,3.007089,116.616667,7.777778
2019-09-27 16:00:00-05:00,24.844167,22.836481,39.083333,181.883333,1.114620,3.171004,234.150000,8.194444
2019-09-27 17:00:00-05:00,20.718981,20.781759,45.066667,15.000000,1.001370,2.670319,129.050000,8.425926


title,Solar Radiation,Temp,RH,Wind Speed,Gust Speed,Wind Direction,Water Content,MRT,DewPt
timezone_str,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,W/mÂ²,Â°C,%,m/s,m/s,Ã¸,mÂ³/mÂ³,Â°C,Â°C
logger_serial_number,20094090,20094090,20094090,20094090,20094090,20094090,20094090,20094090,20094090
sensor_serial_number,20073988,20496044,20496044,20496535,20496535,20496535,20499689,20693833,20496044
datetimes,,,,,,,,,
2019-10-10 13:00:00-05:00,542.700000,17.668704,51.366667,1.114620,2.560049,250.833333,0.057033,24.180648,7.518519
2019-10-10 14:00:00-05:00,433.533333,18.140185,53.033333,1.171245,2.671809,271.000000,0.060233,23.978148,8.444444
2019-10-10 15:00:00-05:00,291.666667,17.811296,54.883333,1.339630,3.228374,293.300000,0.060917,23.008981,8.620370
2019-10-10 16:00:00-05:00,138.533333,16.879722,58.466667,1.451390,4.062103,286.033333,0.059750,20.623148,8.694444
2019-10-10 17:00:00-05:00,12.916667,15.237870,63.650000,0.891100,2.340254,271.000000,0.059250,15.240648,8.388889


In [319]:
# Reindl split test
import time
import math
import pandas as pd
from hoboreader import HoboReader
from gen_reindl import *

S2 = HoboReader("DL2_PSB_Parking_Lot.csv")
df2 = S2.get_dataframe()

# Create date series using date_range() function
date_series = pd.date_range('01/01/2019', periods=8760, freq='H')
# print(date_series)

# altitude M Station (although not necessary for gen_reindl)
altitude = 255

# lat, lon taken from STATION
lat = 42.45062  # north-positive
lon = 76.48120  # west-positive for Reindl split (Alstan)

# Then the gen_reindl program can be run. -l is longitude (west positive),
# -a is latitude (north positive) and -m is the time zone in a multiple of
# 15 degrees from the meridian. The command below is for Singapore, and
# you note that it is in the wrong time zone. UTC+8 * 15 = -120, despite a
# -103.98 longitude.
# > gen_reindl -m -120 -l -103.98 -a 1.37 -i input.txt -o output.wea

# pos
time_zone = 5 * 15  # west-positive

#print("month, day, time, GHR\n")


# %timeit GenReindl(lat, lon, time_zone).calc_split(4, 22, 8.333333333, 107)


def calc_split_df(df):
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['DHR'] = 0
    df['DNI'] = 0
    solar_rad_column = np.argmax(df.columns.get_loc('Solar Radiation'))
    

    gr = GenReindl(lat, lon, time_zone)
    
    
    utc = timezone('UTC')    
    fmt = '%Y-%m-%d %H:%M:%S %Z%z'

    length = df.shape[0]
    idx = df.index

    for i in tqdm(range(length), total=length):
        GHR = df.iloc[i, solar_rad_column]
        DNI, DHR = gr.calc_split(df.month[i], df.day[i], df.hour[i], GHR)
        #print(i,  GHR, DNI, DHR)
        dt = datetime(year=idx.year[i], month=idx.month[i],
                      day=idx.day[i], hour=idx.hour[i], tzinfo=utc)
        
        #loc_dt = utc_dt.astimezone(eastern)
        #loc_dt.strftime(fmt)
        
        df.loc[dt, 'DHR'] = DHR
        df.loc[dt, 'DNI'] = DNI

    return df


#calc_split_df(df5[:100])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 43.95it/s]


title,#,Date Time,Temp,RH,Wind Speed,Wind Direction,Solar Radiation,Precipitation,DewPt,SkyCover,Gust Speed,hour,day,month,year,DHR,DNI
timezone_str,NaN,GMT-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,
units,NaN,NaN,Â°C,%,m/s,Ã¸,W/mÂ²,Inches,Â°C,Fraction,m/s,,,,,,
logger_serial_number,NaN,NaN,20098520,21098500,20198500,20198500,20034090,20034090,20495520,20075520,20054090,,,,,,
sensor_serial_number,NaN,NaN,10762331,10263331,21706789,21706789,23073988,23073988,10232331,10734331,20896535,,,,,,
datetimes,,,,,,,,,,,,,,,,,
2018-12-31 22:00:00+00:00,1,01/01/19 12:00:00 AM,5.00,96.0,3.639,156.0,0.00,0.06,4.20,1,6.093,22,31,12,2018,0.00,0.0
2018-12-31 23:00:00+00:00,2,01/01/19 01:00:00 AM,5.83,95.0,3.518,156.0,1.16,0.00,4.83,1,6.773,23,31,12,2018,1.16,0.0
2019-01-01 00:00:00+00:00,3,01/01/19 02:00:00 AM,6.72,95.0,2.418,155.0,1.16,0.00,5.72,1,4.318,0,1,1,2019,1.16,0.0
2019-01-01 01:00:00+00:00,4,01/01/19 03:00:00 AM,7.67,94.0,2.660,153.0,0.00,0.00,6.47,0.8,4.198,1,1,1,2019,0.00,0.0
2019-01-01 02:00:00+00:00,5,01/01/19 04:00:00 AM,8.67,93.0,1.699,161.0,0.00,0.00,7.27,1,3.818,2,1,1,2019,0.00,0.0


# Variables in the epw that we need


# From GFR
average temperature (F),
 relative humidity (%), 
solar radiation (langleys), 
10 meter wind speed (mph), 
wind direction (degrees), 
precipitation (inches)

# To EPW

average temperature (°C),
relative humidity (%), 
GHI  (Wh/m2), 
DNI  (Wh/m2), 
DHI  (Wh/m2), 
10 meter wind speed (m/s), 
wind direction (degrees), 
precipitation (cm or mm)


# Make the 2019 file
# Make the 2020 file and replace half of the year with the values from 2019

 df5_2020[start_time:end_time] = df5_2019.loc[start_time:end_time]



In [325]:
def get_1y_data(df, year):
    utc = timezone('UTC')
    #start = datetime(int(year), 1, 1, 0, 0, tzinfo=utc)
    #end = datetime(int(year), 12, 31, 23, 0, tzinfo=utc)
    
    dt = pd.date_range(datetime(int(year), 1, 1, 0, 0),
                       datetime(int(year), 12, 31, 23, 0),
                       freq="60min", tz = utc)
    
    df.resample('H')
    df3 = df.reindex(dt, fill_value=9999)   

    return df3


# get_1y_data(df5,2019)


def make_epw(title, year, hoho_df, reindl_df):

    e = epw()

    e.headers = {
        'LOCATION': [
            'Ithaca', 'NY', 'USA', 'TMY3', '725190', '42.43', '-76.49', '-5.0',
            '125.0'
        ],
        'DESIGN CONDITIONS': [],
        'TYPICAL/EXTREME PERIODS': [],
        'GROUND TEMPERATURES': [],
        'HOLIDAYS/DAYLIGHT SAVINGS': [],
        'COMMENTS 1': [],
        'COMMENTS 2': [],
        'DATA PERIODS': []
    }

    # print(e.headers)

    #variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPoint" ]

    dt = pd.date_range(datetime(int(year), 1, 1, 0, 0),
                       datetime(int(year), 12, 31, 23, 0),
                       freq="60min")

    if calendar.isleap(year):
        missing_values = np.array(np.ones(8784) * 999999).astype(int)
    else:
        missing_values = np.array(np.ones(8760) * 999999).astype(int)

    edf = e.dataframe

    edf['Year'] = dt.year.astype(int)
    edf['Month'] = dt.month.astype(int)
    edf['Day'] = dt.day.astype(int)
    edf['Hour'] = dt.hour.astype(int) + 1
    edf['Minute'] = dt.minute.astype(int)
    edf['Data Source and Uncertainty Flags'] = missing_values

    # Actual file starts here
    # 6
    # .apply(lambda x: ftoc(x), axis=1).values.flatten()
    edf['Dry Bulb Temperature'] = hoho_df['Temp'].values.flatten()

    #     Add this later (ADDED TO DL5_2019 FILE 07.22.2020)
    # missing_values
    edf['Dew Point Temperature'] = hoho_df['DewPt'].values.flatten()

    edf['Relative Humidity'] = hoho_df['RH'].values.flatten()

    # Added NREL pressure 2020-08-21
    if 'Pressure' in hoho_df:
        edf['Atmospheric Station Pressure'] = hoho_df['Pressure'].values.flatten()
    else:
        edf['Atmospheric Station Pressure'] = missing_values

    edf['Extraterrestrial Horizontal Radiation'] = missing_values
    #
    edf['Extraterrestrial Direct Normal Radiation'] = missing_values
    #
    edf['Horizontal Infrared Radiation Intensity'] = missing_values
    #
    edf['Global Horizontal Radiation'] = hoho_df[
        'Solar Radiation'].values.flatten()
    edf['Direct Normal Radiation'] = reindl_df['DNI'].values.flatten()
    edf['Diffuse Horizontal Radiation'] = reindl_df['DHR'].values.flatten()

    # Do we need this?
    # https://www.radiance-online.org//pipermail/radiance-general/2013-January/009140.html ?
    edf['Global Horizontal Illuminance'] = hoho_df['Solar Radiation'].values.flatten() * \
        179
    edf['Direct Normal Illuminance'] = reindl_df['DNI'].values.flatten() * 179
    edf['Diffuse Horizontal Illuminance'] = reindl_df['DHR'].values.flatten(
    ) * 179

    edf['Zenith Luminance'] = missing_values

    edf['Wind Direction'] = hoho_df['Wind Direction'].values.flatten().astype(
        int)
    edf['Wind Speed'] = hoho_df['Wind Speed'].values.flatten()

    if 'SkyCover' in hoho_df:

        # not sure which one to use (skycover)
        edf['Total Sky Cover'] = hoho_df['SkyCover'].replace(
            ' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
    #     This is from Ithaca airport data (NWS, NOAA) (ADDED TO DL5_2019 FILE 07.22.2020)
    #  (used if Horizontal IR Intensity missing)
        edf['Opaque Sky Cover'] = hoho_df['SkyCover'].replace(
            ' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
    else:
        edf['Total Sky Cover'] = missing_values
        edf['Opaque Sky Cover'] = missing_values

    #

    edf['Visibility'] = missing_values
    edf['Ceiling Height'] = missing_values
    edf['Present Weather Observation'] = missing_values
    #
    edf['Present Weather Codes'] = missing_values
    edf['Precipitable Water'] = missing_values
    edf['Aerosol Optical Depth'] = missing_values
    #
    edf['Snow Depth'] = missing_values
    edf['Days Since Last Snowfall'] = missing_values
    edf['Albedo'] = missing_values
    #

    # We have precipitation but lets not use it for now
    edf['Liquid Precipitation Depth'] = missing_values
    edf['Liquid Precipitation Quantity'] = missing_values

    #pd.options.display.max_columns = None
    # display(edf.loc[8:15])

    filename = str(title) + "_" + str(year) + '.epw'

    e.write(filename)

    with open(filename, 'r') as file:
        filedata = file.read()
        filedata = filedata.replace('.0,', ',')
    with open(filename, 'w') as file:
        file.write(filedata)


def show_missing_vals(csv, year):

    S = HoboReader(csv)

    df = S.get_dataframe()
    dfm = df.resample('H').mean()

    df_1y = get_1y_data(dfm, year)
    df_1y.index = df_1y.index.tz_convert(None)

    ix = pd.date_range(datetime(int(year), 1, 1, 0, 0),
                       datetime(int(year), 12, 31, 23, 0),
                       freq="60min")
    df_1y_re = df_1y.reindex(ix)
    fig = msno.matrix(df_1y_re.set_index(ix), freq='M')
    fig_copy = fig.get_figure()
    fig_copy.savefig(s.split('.')[0] + '_' + str(year) +
                     '_MissingVals.png', bbox_inches='tight')

In [361]:
dd = get_1y_data(df5,2020)
dd.iloc[:2000,:]

title,#,Date Time,Temp,RH,Wind Speed,Wind Direction,Solar Radiation,Precipitation,DewPt,SkyCover,Gust Speed
timezone_str,NaN,GMT-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,NaN,NaN,Â°C,%,m/s,Ã¸,W/mÂ²,Inches,Â°C,Fraction,m/s
logger_serial_number,NaN,NaN,20098520,21098500,20198500,20198500,20034090,20034090,20495520,20075520,20054090
sensor_serial_number,NaN,NaN,10762331,10263331,21706789,21706789,23073988,23073988,10232331,10734331,20896535
2020-01-01 00:00:00+00:00,8762,01/01/20 02:00:00 AM,-0.49,72.27,1.873,267.4,0.00,0.00,-6.03,1,4.095
2020-01-01 01:00:00+00:00,8763,01/01/20 03:00:00 AM,-0.69,73.12,1.453,271.2,0.00,0.00,-6.07,1,2.736
2020-01-01 02:00:00+00:00,8764,01/01/20 04:00:00 AM,-0.84,76.88,1.994,261.3,0.00,0.00,-5.46,1,3.344
2020-01-01 03:00:00+00:00,8765,01/01/20 05:00:00 AM,-0.97,78.23,1.775,260.5,0.00,0.00,-5.32,1,2.883
2020-01-01 04:00:00+00:00,8766,01/01/20 06:00:00 AM,-1.22,81.90,1.484,241.9,0.00,0.00,-4.84,1,2.597
...,...,...,...,...,...,...,...,...,...,...,...


In [368]:
dd = get_1y_data(df5,2019)
dd.iloc[25:40,:]

title,#,Date Time,Temp,RH,Wind Speed,Wind Direction,Solar Radiation,Precipitation,DewPt,SkyCover,Gust Speed
timezone_str,NaN,GMT-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,NaN,NaN,Â°C,%,m/s,Ã¸,W/mÂ²,Inches,Â°C,Fraction,m/s
logger_serial_number,NaN,NaN,20098520,21098500,20198500,20198500,20034090,20034090,20495520,20075520,20054090
sensor_serial_number,NaN,NaN,10762331,10263331,21706789,21706789,23073988,23073988,10232331,10734331,20896535
2019-01-02 01:00:00+00:00,28,01/02/19 03:00:00 AM,-3.44,77.0,0.921,336.0,0.00,0.0,-8.04,1,1.779
2019-01-02 02:00:00+00:00,29,01/02/19 04:00:00 AM,-3.56,79.0,0.979,331.0,0.00,0.0,-7.76,1,1.659
2019-01-02 03:00:00+00:00,30,01/02/19 05:00:00 AM,-3.61,79.0,0.800,305.0,0.00,0.0,-7.81,1,1.560
2019-01-02 04:00:00+00:00,31,01/02/19 06:00:00 AM,-3.56,80.0,0.760,316.0,0.00,0.0,-7.56,1,1.600
2019-01-02 05:00:00+00:00,32,01/02/19 07:00:00 AM,-3.78,80.0,0.939,342.0,0.00,0.0,-7.78,1,1.659
2019-01-02 06:00:00+00:00,33,01/02/19 08:00:00 AM,-3.78,81.0,0.559,299.0,1.16,0.0,-7.58,1,0.800


In [365]:
# GAME FARM ROAD_ITHACA STATION

#S5 = HoboReader("DL5_Game_Farm_Road_2019.csv")
#hoho_df = S5.get_dataframe()

# There are about 35 variables in the core weather data. However, not all of them are used by EnergyPlus. Actually, despite of date and time columns, only 13 columns are used:
# dry bulb temperature
# dew point temperature
# relative humidity
# atmospheric pressure
# horizontal infrared radiation intensity from sky
# direct normal radiation
# diffuse horizontal radiation
# wind direction
# wind speed
# present weather observation
# present weather codes
# snow depth
# liquid precipitation depth

# https://www.radiance-online.org/pipermail/radiance-general/2015-April/010841.html

# Daniel,

# You should be able to generate the .wea file from diffuse horizontal
# radiation and direct normal radiation values. The structure of the lines
# after the heading in .wea file is:

# month day hour directNormalRadiation diffuseHorizontalRadiation

# Load NREL 2019 and take atmospheric pressure from there
year = 2019
nrel_2019 = Path.cwd() / Path("nrel_data/1162143_42.45_-76.50_2019.csv")
# print(nrel_2019)
nrel_2019_df = pd.read_csv(nrel_2019)
#df.columns = df.index[0]
nrel_2019_df.columns = nrel_2019_df.loc[nrel_2019_df.index[1]]
nrel_2019_df = nrel_2019_df.drop(nrel_2019_df.index[0])
nrel_2019_df = nrel_2019_df.drop(nrel_2019_df.index[0])

nrel_2019_pressure = nrel_2019_df['Pressure'].astype(int) * 100

# Real file

df5_1y = get_1y_data(df5, 2019)
df5_reindl_1y = calc_split_df(df5_1y)

# e.headers['LOCATION'][0]='Ithaca_NY'

# Design conditions and extreme periods are taken from the Syracuse file

make_epw("GameFarmRoad", 2019,df5_1y, df5_reindl_1y)

100%|██████████████████████████████████████████████████████████████████████████████| 8760/8760 [03:49<00:00, 38.18it/s]


In [362]:
# GAME FARM ROAD_ITHACA STATION


year = 2020


# Real file

df5_1y = get_1y_data(df5, year)

df5_reindl_1y = calc_split_df(df5_1y)

# Design conditions and extreme periods are taken from the Syracuse file

make_epw("GameFarmRoad", year,df5_1y, df5_reindl_1y)

100%|██████████████████████████████████████████████████████████████████████████████| 8784/8784 [04:11<00:00, 34.90it/s]


In [ ]:
# All to EPW
year = [2019, 2020]

stations = ["DL1_Stone_Garden_Tjaden_Hall.csv","DL2_PSB_Parking_Lot.csv","DL3_Olin_Library.csv","DL4_Milstein_Hall_Roof.csv", "DL5_Game_Farm_Road.csv"]

ul= list(itertools.product(year, stations))


for y, s in ul:
    print(y,s)
    
    S = HoboReader(s)
    
    df = S.get_dataframe()
    dfm = df.resample('H').mean()

    df_1y = get_1y_data(dfm,y)
    df_1y.index = df_1y.index.tz_convert(None)
    df_split = calc_split_df(df_1y)

    ix = pd.date_range(datetime(int(y), 1, 1, 0, 0),
                           datetime(int(y), 12, 31, 23, 0),
                           freq="60min")
    df_1y_re= df_1y.reindex(ix).fillna(999)
    df_split_re= df_split.reindex(ix).fillna(999)
    make_epw(s.split('.')[0], y, df_1y_re, df_split_re)

In [ ]:
for y, s in ul:
    print(y,s)   
    show_missing_vals(s, y)

In [ ]:
a = epw()
a.read("DL4_Milstein_2019.epw")
day = 8
month = 12
year = 2019

hour_f = to_hour_of_year(0, day, month, year)
hour_t = to_hour_of_year(23, day, month,year)

print(hour_f, hour_t)

pd.set_option('display.max_rows', None)
a.dataframe.loc[hour_f-48:hour_t+48,['Wind Speed', 'Wind Direction']]
#a.dataframe.loc[:,['Wind Speed', 'Wind Direction']]

#display(a.dataframe)
